In [164]:
import pandas as pd
import re
import os
import json

In [165]:
df = pd.read_csv("unique_artists_5.csv", delimiter=";")

In [166]:
folder = "artist_jsons"

In [167]:
files = os.listdir(folder)

In [168]:
out = list()

In [169]:
jsons = list()

In [220]:
def unpack_json(dic):
    lifespan = dic["life-span"]
    lifespan["life_end"] = lifespan.pop("end")
    lifespan["life_ended"] = lifespan.pop("ended")
    lifespan["life_begin"] = lifespan.pop("begin")
    dic.pop("life-span")
    dic.update(lifespan)
    for field in ["begin_area", "area", "end_area"]:
        if field in dic:
            if dic[field]:
                new_field = dic[field]
                dic.pop(field)
                dic[field] = new_field.pop("name")

In [223]:
rows_list = []
for row in df["artists_names"]:
    clean_filename = re.sub(r"[><:\"?*|\\/]", "", f"{row}.json")
    path = os.path.join(folder, clean_filename)
    base_info = {
        "artists_names":row
        }
    if os.path.isfile(path):
        with open(path, "r") as js:
            json_data = json.load(js)
            unpack_json(json_data)
            base_info.update({"json_filename" : clean_filename})
            base_info.update(json_data)
            rows_list.append(base_info)
    else:
        rows_list.append(base_info)

out = pd.DataFrame(rows_list)       

In [224]:
out.drop(["works", "releases", "release-groups", "aliases", "type-id", "recordings", "gender-id", "begin-area", "end-area"], axis=1).to_csv("artistinfo_almost_final.csv", sep=";", index=False)

In [163]:
missed = out[out['error'].notnull()]
missed["artists_names"].to_csv("missed_timig.csv", sep=";", index=False)

KeyError: 'error'

In [204]:
out['json_filename'].isnull().sum()

47